In [1]:
import skmob
import pandas as pd
import csv, os, sys, json
import urllib.request
from skmob.preprocessing import clustering
from pandas import Series, DataFrame
import math

In [98]:
df = pd.read_csv('./answer.csv')
# print(df)
group = df.groupby('deviceid')
# print(group.first())

prefer = dict()

for key, g in group:
    # print(f"key: {key}")
    prefer[key] = dict()
    for x in g['choices']:
        if str(type(x)) == "<class 'float'>":
            continue
        x = x.split(',{')

        for i, val in enumerate(x):
            # print(f"idx:{idx}, val:{val}")
            if i > 0:
                val = '{ ' + val
            val = val.replace("\'", '\"')
            idx = val.find('index')
            sub = val[idx:idx+8]
            val = val.replace(sub, '\"index\": \"0\"')
            val = val.replace('mappedValue', '\"mappedValue\"')
            val = val.replace('prompt', '\"prompt\"')
            # print(f"val: {val}")
            d = json.loads(val)
            # print(d['mappedValue'])
            if str.isdigit(d['mappedValue']):
                continue
            # print(f"val : {d['mappedValue']}")
            pref = d['mappedValue']
            if pref not in prefer[key]:
                prefer[key][pref] = 1
            else:
                prefer[key][pref] += 1

        # d = json.loads(x)
        # print(f"d: {d}")
# print(prefer)
# new_df = pd.DataFrame(prefer)
# print(new_df)

d2 = sorted(d.items(), key=lambda x: x[1], reverse=True)

for x in prefer:
    prefer[x] = dict(sorted(prefer[x].items(), key=lambda y: y[1], reverse=True))


with open('survey.csv','w') as f:
    w = csv.writer(f)
    # w.writerow(prefer.keys())
    # w.writerow(prefer.values())
    w.writerow(['deviceid', 'poi', 'count'])
    for x in prefer:
        for y in prefer[x]:
            w.writerow([x, y, prefer[x][y]])
